# Step 5: V02 — Secondary eclipse search (phase ∼0.5)

Goal: search for a significant secondary eclipse at orbital phase ∼0.5.

Why this matters:
- A strong secondary eclipse is a common false-positive signature for eclipsing binaries or blends.
- For small planets with shallow transits, we generally expect **no detectable secondary** in the TESS bandpass.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False)
r = session.run('V02')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': r.flags,
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "baseline_flux": 1.000006,
    "n_baseline_points": 21177,
    "n_secondary_events_effective": 8,
    "n_secondary_points": 22130,
    "red_noise_inflation": 4.84,
    "secondary_depth": 9e-06,
    "secondary_depth_err_ppm": 8.43,
    "secondary_depth_ppm": 9.4,
    "secondary_depth_sigma": 1.12,
    "secondary_phase_coverage": 1.0
  },
  "status": "ok"
}
```

</details>


In [ ]:
# Plot V02: Secondary eclipse search
out = {
    'status': r.status,
    'flags': r.flags,
}

try:
    import matplotlib.pyplot as plt
    from tess_vetter.api import plot_secondary_eclipse
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE and (r.status == 'ok') and getattr(r, 'raw', None) and (r.raw or {}).get('plot_data'):
    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='05_v02_secondary')
    run_path = run_out_dir / 'V02_secondary_eclipse.png'
    docs_path = (docs_out_dir / 'V02_secondary_eclipse.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(8, 5))
    plot_secondary_eclipse(r, ax=ax)
    ax.set_title('V02: Secondary eclipse search (phase ~0.5)')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/05_v02_secondary/V02_secondary_eclipse.png`

![V02: Secondary eclipse search](../artifacts/tutorial_toi-5807-incremental/05_v02_secondary/V02_secondary_eclipse.png)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/05_v02_secondary/V02_secondary_eclipse.png",
  "flags": [],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/05_v02_secondary/V02_secondary_eclipse.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** secondary depth ≈ **9.4 ± 8.43 ppm** (**1.12σ**) with full phase coverage.
- **Why it’s useful:** rules out a large, obvious secondary eclipse at phase 0.5 (common EB signature).
- **Interpretation:** consistent with no detectable secondary in TESS.
- **Next step:** V03 (duration consistency / stellar-density sanity check).

</details>
